# Load Model, Tokenizer, dan Dataset

In [4]:
# Load ===================================================================================
from datasets import load_dataset
from utils import load_model_and_tokenizer

model_name = "Qwen/Qwen3-1.7B" 
model, tokenizer, config = load_model_and_tokenizer(model_name)
dataset = load_dataset("khalidrizki/post-retrieval-research_raw-dataset")
print("BERHASIL MELOAD MODEL DAN DATASET\n")

Loading model on cuda with torch.bfloat16


Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00, 15.11it/s]


BERHASIL MELOAD MODEL DAN DATASET



# Formatting

In [ ]:

# Formatting ============================================================================================================
import datasets
from functools import partial

def format_passages(example, psgs_col):
    """
    Join ketiga passages dengan \n\n sebagai penghubung
    """
    
    example['formatted_passages'] = "\n\n".join(example[psgs_col])
    return example

_format_psgs = partial(
    format_passages, 
    psgs_col = 'passages'
)

fin_dataset = {}

for split in dataset.keys():
    fin_dataset[split] = dataset[split].map(_format_psgs)
    fin_dataset[split] = fin_dataset[split].remove_columns('passages')

fin_dataset = datasets.DatasetDict(fin_dataset)

import datasets
from functools import partial

def format_passages(example, psgs_col):
    """
    Join ketiga passages dengan \n\n sebagai penghubung
    """
    
    example['formatted_passages'] = "\n\n".join(example[psgs_col])
    return example

_format_psgs = partial(
    format_passages, 
    psgs_col = 'passages'
)

fin_dataset = {}

for split in dataset.keys():
    fin_dataset[split] = dataset[split].map(_format_psgs)
    fin_dataset[split] = fin_dataset[split].remove_columns('passages')

fin_dataset = datasets.DatasetDict(fin_dataset)
print("BERHASIL MEMFORMAT PASSAGES\n")


# Generate Summaries

In [ ]:
# Generate draft summary =============================================================================================
from summarize import generate_summary_dataset
import time
from datasets import DatasetDict

# Dictionary untuk menyimpan hasil per split
processed_splits = {}

# Loop untuk setiap split (train, dev, test)
for split in fin_dataset.keys():
    print(f"🔄 Memproses split: {split}")

    start = time.time()

    # Jalankan rangkuman untuk split tertentu
    final_dataset = generate_summary_dataset(
        dataset=fin_dataset[split],  # Proses per split
        query_col="query",
        psgs_col="formatted_passages", 
        model=model, 
        tokenizer=tokenizer,
        batch_size=1, 
        temperature=0, 
    )

    end = time.time()
    duration = end - start
    print(f"✅ Split {split} selesai dalam {duration:.2f} detik")

    # Simpan hasil per split ke dalam dictionary
    processed_splits[split] = final_dataset

# Gabungkan kembali hasil per split menjadi DatasetDict
dataset_with_draft_summary = DatasetDict(processed_splits)
print("BERHASIL MEMBUAT DRAFT SUMMARY\n")


In [3]:
# Path penyimpanan hasil akhir
save_path = "generated_data/draft_summary_dataset_with_do_sample_False"

# Simpan dataset yang telah digabungkan
dataset_with_draft_summary.save_to_disk(save_path)

Saving the dataset (1/1 shards): 100%|██████████| 565/565 [00:00<00:00, 51965.48 examples/s]


# Answer Generation with Summary

In [ ]:
from datasets import load_from_disk, DatasetDict, Dataset
from utils import load_model_and_tokenizer

# loaded_dataset = load_from_disk("./generated_data/draft_summary_dataset")
# model_name = "Qwen/Qwen3-1.7B"
# model, tokenizer, config = load_model_and_tokenizer(model_name)

c:\Users\LENOVO\anaconda3\envs\recomp\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading model on cuda with torch.bfloat16


Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00, 16.42it/s]


In [ ]:
# Generate final summary for Fine-Tuning ==================================================================================
from generate_answer import generate_answers_and_compare_between_with_and_without_summary
from datasets import DatasetDict, Dataset
import time

processed_splits = {}

# 🔹 Mulai proses evaluasi per split
for split in dataset_with_draft_summary.keys():
    print(f"🔄 Memproses split: {split}")

    start_time = time.time()
    final_dataset = generate_answers_and_compare_between_with_and_without_summary(
        dataset=dataset_with_draft_summary[split],
        model=model,
        tokenizer=tokenizer
    )
    end_time = time.time()

    # 🕒 Waktu eksekusi
    print(f"✅ Proses selesai dalam {end_time - start_time:.2f} detik")
    processed_splits[split] = Dataset.from_list(final_dataset)

fin_dataset = DatasetDict(processed_splits)

save_path = "./generated_data/RECOMP_tuning_w_do_sample_False_and_wo_Judul_nTeks"
fin_dataset.save_to_disk(save_path)

print(f"📄 Hasil telah disimpan dalam {save_path}")

In [ ]:
save_path = "./generated_data/RECOMP-tuning"
ds = fin_dataset.remove_columns(['summary','generated_results'])
ds.save_to_disk(save_path)